In [ ]:
# Load packages
import pickle
import pandas as pd
import numpy as np
import re, tldextract, time
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from collections import Counter
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.preprocessing import StandardScaler


# ask about filling donation NaN with zero's?

# fill donation NaN with zero based on max years the original data set covers
# original data set goes up to 2022, so fill nan until then


In [ ]:
df = pd.read_stata('FundingConfuciousFedshare.dta')

enrollment = pd.read_csv('Enrollment Data_1-3-2024.csv')



# data comes from here https://sites.ed.gov/foreigngifts/files/2023/11/Section-117_Public_Records_complete_through_2022-10-17.xlsx

In [ ]:
enrollment_list = list(enrollment.columns)[2:-1]



In [ ]:
enrollment_long = pd.melt(enrollment, id_vars=['UnitID','Institution Name'], value_vars=enrollment_list, var_name='year', value_name='Enrollment')

enrollment_long['year'] = enrollment_long['year'].str.replace(r'\D', '', regex=True).astype(int)



In [ ]:
df.dtypes


df['contractstartdate'][1].year

contract_year=[]

for i in df['contractstartdate']:
    contract_year.append(i.year)

df['contract_year']= contract_year



In [ ]:
int(df['date_opened'][3].year)






df['receipt_year'] = df['receiptdate'].dt.year
pd.set_option('display.max_columns', 500)

df['opeid'] = df['opeid'].replace(r'^\s*$', np.nan, regex=True)

df[df['opeid'].isna()]



df[df['amount']==0]

df.sort_values(['receipt_year'], ascending=[False])


df[df['receipt_year']==2022]



In [ ]:
len(df[df['receipt_year'].notna()])- len(df[df['contract_year'].notna()])



In [ ]:
schools = df[['schoolname', 'state']].drop_duplicates()



years = [y for y in range(1995, 2022 + 1)]



years_df = pd.DataFrame(years)


school_years  = schools.merge(years_df, how='cross')

school_years = school_years.rename(columns={0: "year"})



df_school_year =  school_years.merge(df, how='left', left_on=['schoolname', 'state', 'year'], right_on=['schoolname', 'state', 'receipt_year'] )


df_school_year['amount']= df_school_year['amount'].fillna(0)

df_school_year['year'] = df_school_year['year'].astype(float)



#df_school_year_transactions = df_school_year[df_school_year['receipt_year'].notna()]

df_school_year['china'] = df_school_year['china'].fillna(0)

df_school_year['china_donation_bin_sum'] = df_school_year.groupby(['schoolname', 'state', 'year'])['china'].transform('sum')
# if greater than zero mean it has at least one china donatoin


df_school_year['china_donation'] = np.where(df_school_year['china_donation_bin_sum']>0, 1, 0)


df_school_year['total_foreign_donation'] = df_school_year.groupby(['schoolname', 'state', 'year'])['amount'].transform('sum')

df_school_year['mean_foreign_donation'] = df_school_year.groupby(['schoolname', 'state', 'year'])['amount'].transform('mean')

In [ ]:
#df_school_year_transactions= df_school_year_transactions[['schoolname', 'state', 'year', 'china', 'total_foreign_donation','mean_foreign_donation' ,'china_donation']]



In [ ]:
df_school_year[df_school_year['schoolname']=='Gill-Tech Academy of Hair Design']


df_school_year[df_school_year['schoolname']=='Lincoln University']


In [ ]:
df_receipt = df_school_year.dropna(subset=['receipt_year'])
df_no_receipt = df_school_year[df_school_year['receipt_year'].isnull()]


# fix this variable by using receiptdate year
df_receipt['year_match'] = np.where(df_receipt["year"] == df_receipt["receipt_year"], True, False)

df_receipt_clean = df_receipt[df_receipt['year_match']==True] 


df_receipt_clean = df_receipt_clean.drop('year_match', axis=1)


df_donations = pd.concat([df_receipt_clean, df_no_receipt])


df_donations[df_donations['schoolname']=='Arizona State University']['year'].unique()


In [ ]:


CI_open_year = []

CI_close_year = []

for i in df_donations['date_opened']:
    if pd.notna(i):
        CI_open_year.append(int(i.year))
    else:
        CI_open_year.append(i)
    
for i in df_donations['date_closed']:
    if pd.notna(i):
        CI_close_year.append(int(i.year))
    else:
        CI_close_year.append(i)

        
        
df_donations['CI_open_year'] = CI_open_year

df_donations['CI_close_year'] = CI_close_year


CI_open_years = []

for i, j in zip(df_donations['CI_open_year'], df_donations['CI_close_year']):
    # change code so it does if else for three variations: NAN for both lists, NAN for only close, and no NAN
    if pd.notna(i) &  pd.notna(j):
        CI_open_years.append( [y for y in range(i, j + 1)])
        # modify this code to get years for Confucius Institutes still open
    elif pd.notna(i) & pd.isna(j):
        CI_open_years.append( [y for y in range(i, 2023 + 1)])
    
    else:
        CI_open_years.append(j)





#CI variable is a bianry for if they EVER had Confucious institute
# CI_open is a binary variable for if they have a confucious institute open this year


# change CI_open_years to include years for Confucius institutes that never closed





In [ ]:
#group and aggregate donations by year and university
# make each row a university year combination

df_donations['CI_open_years'] = CI_open_years

df_merged = df_donations.explode('CI_open_years')




df_merged[df_merged['schoolname']=='Arizona State University']['year'].unique()


CI_open = []

for i, j, k in zip(df_merged['CI_open_years'], df_merged['receipt_year'], df_merged['year']):
    # change code so it does if else for three variations: NAN for both lists, NAN for only close, and no NAN
    if (i == j) & (i == k) & pd.notna(i):
        CI_open.append(1)
    elif (i != j) & (i == k):
        CI_open.append(1)
    elif (i != j) & (i != k) & (j == k):
        CI_open.append(0)
    else:
        CI_open.append(i)


df_merged['CI_open'] = CI_open

df_merged[(df_merged['schoolname']=='Arizona State University')&(df_merged['year']==2009)]







In [ ]:
df_merged[(df_merged['schoolname']=='Stanford University')]


In [ ]:

df_clean = df_merged.sort_values('CI_open', ascending=False).drop_duplicates(['schoolname', 'state', 'year']).sort_index()


df_clean = df_clean.drop(columns = ['china'])

#df_clean = df_clean.merge(df_school_year_transactions, how='left', left_on=['schoolname', 'state', 'year'], right_on=['schoolname', 'state', 'year'] )





df_clean[(df_clean['schoolname']=='Arizona State University')]


In [ ]:

#df_school_year = df_school_year.drop(columns=['year'])

#df_school_year_clean =  school_years.merge(df_school_year, how='left', left_on=['schoolname', 'year'], right_on=['schoolname', 'CI_open_years'] )



In [ ]:
#df_school_year_clean['CI_open'] =  (df_school_year_clean["CI_open_years"].notna()) * 1



In [ ]:

# change CI_open_years to NAN for columns where receipt year does not have match with CI_open_years

In [ ]:



# drop rows where academic year does not match CI_open_years


# to do this, drop all nan for CI_open_years in a seperate data frame for confucius institute universities
# also make seperate data frame with only NaN CI_open_years values
# check matches for academic year and CI_open_years in new data frame
# append two data frames back together


#df_CI = df_merged.dropna(subset=['CI_open_years'])
#df_no_CI = df_merged[df_merged['CI_open_years'].isnull()]

#df_CI['CI_open_years'] = df_CI['CI_open_years'].astype(float)

# fix this variable by using receiptdate year
#df_CI['year_match'] = np.where(df_CI["year"] == df_CI["CI_open_years"], True, False)




#df_CI_clean = df_CI[df_CI['year_match']==True] 


#df_CI_clean = df_CI_clean.drop('year_match', axis=1)

#df_clean = pd.concat([df_CI_clean, df_no_CI])

#df_clean

# fix this with outer merge

#df_clean[(df_clean['CI_open']==1) ]


#df_CI_clean[df_CI_clean['schoolname']=='Arizona State University']['year'].unique()


#df_clean = df_donations.merge(df_confucius, how='cross')




In [ ]:










df_clean[df_clean['year']==1995]

df_clean[df_clean['schoolname']=='Arizona State University']






In [ ]:



# make variable for total china donation


df_china_donations = df_school_year[df_school_year['china']==1]

df_china_donations['total_china_donation'] = df_china_donations.groupby(['schoolname', 'state', 'year'])['amount'].transform('sum')



df_china_donations['mean_china_donation'] = df_china_donations.groupby(['schoolname', 'state', 'year', 'china'])['amount'].transform('mean')

# combine china donation data frame back to clean data frame to get china stats


df_china_donations_filtered = df_china_donations[['schoolname', 'state', 'year', 'total_china_donation','mean_china_donation']]


master_df = df_clean.drop_duplicates(subset=['schoolname', 'state', 'year'], keep="first")



master_df = master_df.merge(df_china_donations_filtered, how = 'left', on = ['schoolname', 'state', 'year'])

master_df['mean_fed_share'] = master_df.groupby(['schoolname', 'state', 'receipt_year'])['federal_share'].transform('mean')

master_df[(master_df['CI_open_years'].isna())&(master_df['receipt_year'].notna())]




df_china_donations[df_china_donations['schoolname']=="Arizona State University"]





In [ ]:
constants = df

constants = constants[['schoolname', 'state', 'public', 'federal10', 'tot_rev_wo_auxother_sum', 'r', 'federal_share',  'govt_reliance_a']]

constants = constants.drop_duplicates()


constants[constants['schoolname']=='Lincoln University']





In [ ]:
df[(df['schoolname']== 'Columbia College')&(df['state']== 'MO')]



In [ ]:
master_df = master_df[['schoolname', 'state',  'year', 'CI', 'CI_open', 'china_donation', 'total_foreign_donation', 'mean_foreign_donation','total_china_donation', 'mean_china_donation']]




master_df =  master_df.merge(constants, how='left', left_on=['schoolname', 'state'], right_on=['schoolname', 'state'] )




In [ ]:
master_df[master_df['schoolname']=='Alabama A&M University']


master_df[master_df['schoolname']=="Saint John's University"]



zero_china_donation = master_df[master_df['total_foreign_donation'].notna()]

zero_china_donation['total_china_donation'] = zero_china_donation['total_china_donation'].fillna(0)

zero_china_donation['mean_china_donation'] = zero_china_donation['mean_china_donation'].fillna(0)

zero_china_donation=zero_china_donation[['schoolname', 'state', 'year', 'total_china_donation','mean_china_donation']]

master_df = master_df.drop(columns = ['total_china_donation','mean_china_donation'])


master_df = master_df.merge(zero_china_donation, how='left', left_on=['schoolname', 'state', 'year'], right_on=['schoolname', 'state', 'year'] )



In [ ]:
master_df[master_df['schoolname']=='Arizona State University']




In [ ]:


master_df = master_df.sort_values(by=['schoolname', 'state', 'year'])




master_df['schoolname'] = master_df['schoolname'].astype(str)



master_df['state'] = master_df['state'].astype(str)

master_df['CI_open'] = master_df['CI_open'].astype(str)


master_df = master_df.drop_duplicates(subset=['schoolname', 'state', 'year'], keep="first")



master_df


In [ ]:
master_df[master_df['schoolname']=="Arizona State University"]

transition= df_donations
transition['CI_open_years']= CI_open_years

transition = transition[transition['CI_open_years'].notna()]

transition = transition[['schoolname', 'state', 'CI_open_years']]


transition = transition.explode('CI_open_years')


transition['CI_open']= 1



master_df_clean = master_df.drop(columns=['CI_open'])


master_df_clean = master_df_clean.merge(transition, how = 'left',  left_on=['schoolname', 'state', 'year'], right_on=['schoolname', 'state', 'CI_open_years'])


master_df_clean = master_df_clean.drop_duplicates()


master_df_clean['CI_open'] = master_df_clean['CI_open'].fillna(0)



 
master_df_clean = master_df_clean.rename(columns={'CI': "CI_ever"})


master_df_clean['CI_open'] = master_df_clean['CI_open'].astype(int)


master_df_clean['year'] = master_df_clean['year'].astype(int)



master_df_clean['CI_ever'] = master_df_clean.groupby(['schoolname', 'state'])['CI_ever'].fillna(method='bfill', axis=0).fillna(method='ffill', axis=0)


total_foreign_donation = []

for i, j in zip(master_df_clean['total_foreign_donation'], master_df_clean['mean_foreign_donation']):
    if pd.isna(j):
        total_foreign_donation.append(np.nan)
    else:
        total_foreign_donation.append(i)


master_df_clean = master_df_clean.drop(columns=['total_foreign_donation'])
master_df_clean['total_foreign_donation'] = total_foreign_donation

master_df_clean = master_df_clean[['schoolname', 'state',  'public', 'year', 'CI_ever', 'CI_open', 'r', 'china_donation', 'federal10', 'tot_rev_wo_auxother_sum', 'federal_share', 'govt_reliance_a',  'total_foreign_donation', 'mean_foreign_donation','total_china_donation', 'mean_china_donation']]










In [ ]:

# write up data description 
# make sure there are no sampling problems. Count number of private 
# look at characteristics for universities that take tons of China money that didn't set up a CI

In [ ]:
CI_survival_years = master_df_clean[master_df_clean['CI_open']==1]




CI_survival_years['CI_survival_years'] = CI_survival_years.groupby(['schoolname', 'state'])['CI_open'].cumsum()
CI_survival_years = CI_survival_years[['schoolname', 'state', 'year', 'CI_survival_years']]


master_df_clean = master_df_clean.merge(CI_survival_years, how = 'left',  left_on=['schoolname', 'state', 'year'], right_on=['schoolname', 'state', 'year'])


# CI institute survival by years

No_CI_years = master_df_clean[master_df_clean['CI_open']==0]


No_CI_years['No_CI_True'] = 1

No_CI_years['No_CI_years'] = No_CI_years.groupby(['schoolname', 'state'])['No_CI_True'].cumsum()
No_CI_years = No_CI_years[['schoolname', 'state', 'year', 'No_CI_years']]


master_df_clean = master_df_clean.merge(No_CI_years, how = 'left',  left_on=['schoolname', 'state', 'year'], right_on=['schoolname', 'state', 'year'])




In [ ]:
master_df_clean[master_df_clean['schoolname']=='Arizona State University']

In [ ]:

# create variable for survival analysis

In [ ]:

master_df_clean[master_df_clean['schoolname']=='Alabama A&M University']


master_df_clean[master_df_clean['schoolname']=='Alabama A&M University']



constants

master_df_clean[master_df_clean['schoolname']=="Saint John's University"]

constants[constants['schoolname']=="Saint John's University"]



In [ ]:
master_df_clean[master_df_clean['schoolname']=='Arizona State University']

duplicate_rows = master_df_clean[master_df_clean.duplicated()]


master_df_clean[master_df_clean['total_foreign_donation']==0]

df[df['schoolname']=='Gill-Tech Academy of Hair Design']






len(master_df_clean[master_df_clean['schoolname']=='Arizona State University'])




master_df_clean[master_df_clean['schoolname']=="Saint John's University"]


master_df_clean[master_df_clean['schoolname']=='Columbia College']



In [ ]:
from collections import Counter



master_df_clean[master_df_clean['schoolname'].isin(['Columbia College','Lincoln University', 'Bryant & Stratton Colleg','MBTI Business Training Institute', "Saint John's University"])]

master_df_clean[master_df_clean['schoolname']=="Saint John's University"]
df[df['schoolname']=="Saint John's University"]


master_df_clean['university_state'] = master_df_clean['schoolname']+ '_'+master_df_clean['state']


master_df_clean = master_df_clean[['university_state', 'schoolname', 'state',  'public', 'year', 'CI_ever', 'CI_open', 'No_CI_years', 'CI_survival_years', 'r', 'china_donation', 'federal10', 'tot_rev_wo_auxother_sum', 'federal_share', 'govt_reliance_a',  'total_foreign_donation', 'mean_foreign_donation','total_china_donation', 'mean_china_donation']]






In [ ]:

len(master_df_clean['schoolname'].unique())

In [ ]:
master_df_clean[master_df_clean['schoolname']=='Arizona State University']

master_df_clean[master_df_clean['schoolname']=='Gill-Tech Academy of Hair Design']

# change code so that it automates output of graphs
master_df_clean[master_df_clean['schoolname']=='Georgetown University']

master_df_clean[master_df_clean['schoolname']=='Arizona State University']



In [ ]:
master_df_clean[master_df_clean['schoolname']=='Columbia College']['university_state'].unique()
master_df_clean[master_df_clean['schoolname']=='Lincoln University']['university_state'].unique()
master_df_clean[master_df_clean['schoolname']=='Bryant & Stratton College']['university_state'].unique()
master_df_clean[master_df_clean['schoolname']=='MBTI Business Training Institute']['university_state'].unique()
master_df_clean[master_df_clean['schoolname']=='Marian University']['university_state'].unique()


Counter(list(master_df_clean['schoolname']))


master_df_clean[master_df_clean['schoolname']=="Saint John's University"]['university_state'].unique()



In [ ]:
empty_university_with_error = df[df['CI'].isna()][['schoolname', 'state']]

empty_university_with_error['university_state'] = empty_university_with_error['schoolname']+'_'+empty_university_with_error['state']

empty_university_with_error = list(empty_university_with_error['university_state'])



In [ ]:
master_df_clean[master_df_clean['schoolname']=='Columbia College']['university_state'].unique()

public = ['Columbia College_CA', 'Lincoln University_MO', 'Lincoln University_PA']

private = ['Columbia College_MO', 'Columbia College_SC', 'Columbia College_VA', 'Lincoln University_CA', 'Bryant & Stratton College_NY', 'Bryant & Stratton College_OH', 'MBTI Business Training Institute_PR', 'MBTI Business Training Institute_WI', 'Marian University_IN', 'Marian University_WI', "Saint John's University_MN","Saint John's University_NY" ]


master_df_clean.loc[(master_df_clean['university_state'].isin(public)), 'public'] = 1

master_df_clean.loc[(master_df_clean['university_state'].isin(private)), 'public'] = 0


master_df_clean[master_df_clean['university_state']== 'Lincoln University_CA']



master_df_clean = master_df_clean[~master_df_clean["university_state"].isin(empty_university_with_error)]




In [ ]:
demographics = pd.read_excel('tabn310.20.xlsx')

In [ ]:
demographics.dtypes

demographics[demographics.columns[0]]

ind_list = [0, 3, 12]
demographics = demographics.iloc[ind_list].dropna(axis=1)
demographics.columns = demographics.iloc[0]

demographics= demographics.iloc[1: , :]

demographics_china= demographics.iloc[1: , :]

demographics_total = demographics.drop([12],axis=0)

demographics_china_long = demographics_china.melt()

demographics_total_long = demographics_total.melt()

demographics_china_long.columns = demographics_china_long.iloc[0]

demographics_total_long.columns = demographics_total_long.iloc[0]

demographics_china_long= demographics_china_long.iloc[1: , :]


demographics_total_long= demographics_total_long.iloc[1: , :]

demographics_china_long = demographics_china_long.rename(columns={"Continent, region, and selected countries of origin": "demographic_years", '    China': "china_students_in_USA"})


demographics_total_long = demographics_total_long.rename(columns={"Continent, region, and selected countries of origin": "demographic_years", '     Total ' : "total_foreign_students"})




In [ ]:
demographics_clean = demographics_china_long.merge(demographics_total_long)

demographics_clean['china_student_percent'] = (demographics_clean['china_students_in_USA']/demographics_clean['total_foreign_students'])*100

start_school_year= []

end_school_year= []

for i in demographics_clean['demographic_years']:
    start_school_year.append(int(i[0:4]))
    end_school_year.append(int(i[0:2]+i[5:7]))

demographics_clean['year'] = start_school_year

demographics_clean['end_school_year'] = end_school_year

# add total foreign

demographics_for_merge = demographics_clean[['year', 'china_students_in_USA', 'total_foreign_students', 'china_student_percent']]

demographics_for_merge

In [ ]:
master_df_clean = master_df_clean.merge(demographics_for_merge, how='left', left_on=['year'], right_on=['year'])


master_df_clean['china_students_in_USA'] = master_df_clean['china_students_in_USA'].astype(float)
master_df_clean['total_foreign_students'] = master_df_clean['total_foreign_students'].astype(float)
master_df_clean['china_student_percent'] = master_df_clean['china_student_percent'].astype(float)



In [ ]:
master_df_clean['china_donation_proportion']= master_df_clean['total_china_donation']/master_df_clean['tot_rev_wo_auxother_sum']


master_df_clean['china_donation_over_federal']= master_df_clean['total_china_donation']/master_df_clean['federal10']



In [ ]:
# create cumulative china donation variable

master_df_clean['cumulative_china_donations'] = master_df_clean.groupby('university_state')['total_china_donation'].cumsum()


master_df_clean[master_df_clean['schoolname']=='University of Arizona (The)']

In [ ]:


master_df_enroll = master_df_clean.merge(enrollment_long, how='left', left_on=['schoolname', 'year'], right_on=['Institution Name','year'])



In [ ]:





initial_data = master_df_enroll[['UnitID','university_state', 'schoolname', 'year', 'CI_open',  'public', 'Enrollment', 'federal_share' , 'total_foreign_donation', 'total_china_donation' ]]

initial_data[initial_data['schoolname']=='Harvard University']



# create labels for variables

initial_data[initial_data['UnitID'].isna()]

enrollment_long[enrollment_long['Institution Name']== 'Adelphi University']



(len(initial_data[(initial_data['year']!=2022)&(initial_data['UnitID'].isna())]))/len(initial_data)




initial_data[initial_data['schoolname']=='Yale University']

print (len(initial_data[initial_data['UnitID'].isna()]))
print (len(initial_data[initial_data['UnitID'].notna()]))

len(initial_data[(initial_data['year']!=2022)&(initial_data['UnitID'].isna())]['university_state'].unique())


len(initial_data[(initial_data['UnitID'].notna())]['university_state'].unique())

In [ ]:


df[df['schoolname']=='Yale University']

initial_data[initial_data['UnitID'] == 130794]

print(100*(initial_data[initial_data['public']==1]['federal_share'].max()))

print(100*(initial_data[initial_data['public']==1]['federal_share'].min()))

print(100*(initial_data[initial_data['public']==0]['federal_share'].max()))

print(100*(initial_data[initial_data['public']==0]['federal_share'].min()))




In [ ]:
print((initial_data[initial_data['public']==1]['total_china_donation'].max()))

print((initial_data[initial_data['public']==1]['total_china_donation'].min()))

print((initial_data[initial_data['public']==0]['total_china_donation'].max()))

print((initial_data[initial_data['public']==0]['total_china_donation'].min()))


In [ ]:
initial_data.to_stata('initial_data.dta')  


In [ ]:
test_val = master_df_clean[['schoolname','state']]
test_val = test_val.drop_duplicates()






print(len(test_val)*len(master_df_clean[master_df_clean['schoolname']=='Arizona State University']))

len(master_df_clean)


In [ ]:
# modify this code with new data
survival_df = master_df_enroll[master_df_enroll['year']<2020]

survival_df['CI_opening'] = survival_df['CI_open'].diff()


# put experiment cut off year at 2020 based on Trump crackdown https://www.pacificresearch.org/trump-administration-cracks-down-on-chinas-u-s-education-propaganda/

survival_df[survival_df['schoolname']=='Arizona State University']






survival_df['all_year_china_donations'] = survival_df.groupby('university_state')['total_china_donation'].transform('sum')



survival_df['all_year_foreign_donations'] = survival_df.groupby('university_state')['total_foreign_donation'].transform('sum')

survival_df['all_year_federal10'] = survival_df.groupby('university_state')['federal10'].transform('sum')



survival_df['all_year_tot_rev_wo_auxother_sum'] = survival_df.groupby('university_state')['tot_rev_wo_auxother_sum'].transform('sum')






survival_df = survival_df[['university_state','schoolname', 'state','public', 'Enrollment', 'year','CI_ever', 'No_CI_years','CI_opening', 'CI_survival_years', 'federal_share', 'total_china_donation', 'cumulative_china_donations','all_year_federal10','all_year_tot_rev_wo_auxother_sum','all_year_china_donations','all_year_foreign_donations']]

survival_df_no_CI = survival_df[survival_df['CI_ever']==0]

survival_df_no_CI= survival_df_no_CI[survival_df_no_CI['year']==2019]


# if status = 0, then the data was censored
survival_df_no_CI['status'] = 0


survival_df_got_CI = survival_df


survival_df_got_CI['No_CI_years']  = survival_df_got_CI.groupby('university_state')['No_CI_years'].ffill()


survival_df_got_CI = survival_df[survival_df['CI_opening']==1]
survival_df_got_CI['status'] = 1

survival_df_clean = pd.concat([survival_df_got_CI, survival_df_no_CI])




In [ ]:

survival_df[survival_df['university_state']=='University of Oregon_OR']



In [ ]:
len(survival_df_clean[survival_df_clean['all_year_china_donations']>0])
len(survival_df_clean[survival_df_clean['all_year_china_donations']==0])

In [ ]:

len(survival_df['university_state'].unique())


len(survival_df_clean['university_state'].unique())




len(survival_df_no_CI[survival_df_no_CI['year']==2019])

len(survival_df[survival_df['CI_ever']==0]['university_state'].unique())



In [ ]:
len(survival_df_clean)


len(survival_df_clean['university_state'].unique())


In [ ]:
# start doing descriptive statistics based on Christine's requests

# descriptive Stats

# Number of schools 

from matplotlib import pyplot as plt


len(initial_data['university_state'].unique())


initial_data[initial_data['CI_open']==1]['university_state'].unique()



# Number of schools by CI ever

bar_df = survival_df_clean.groupby('CI_ever').count().reset_index()

bar_df = bar_df.rename(columns={"university_state": "Number of Schools"})



bar_df['CI_ever'] = bar_df['CI_ever'].map({0: 'No CI Ever', 1: 'Had a CI'})


x = bar_df['CI_ever']
y = bar_df['Number of Schools']

fig, ax = plt.subplots()
bars = ax.bar(x, y)

ax.bar_label(bars)

plt.ylabel("Count")



plt.title('Distibution of Universities that ever had a CI (1999-2019)')

fig.figure.savefig('CI_ever_bar.png', bbox_inches='tight')



In [ ]:

import seaborn as sns

bar_df = survival_df_clean


bar_df = bar_df.groupby(['CI_ever', 'public']).count().reset_index()



bar_df = bar_df.rename(columns={"university_state": "Number of Schools"})



bar_df['CI_ever'] = bar_df['CI_ever'].map({0: 'No CI Ever', 1: 'Had a CI'})


x = bar_df['CI_ever']
y = bar_df['Number of Schools']
color = bar_df['public']

ax = sns.barplot(data=bar_df, x=x, y=y, hue=color)


for i in ax.containers:
    ax.bar_label(i,)


plt.title('Distibution of Universities that ever had a CI by Public or Privte')






plt.savefig('CI_ever_pub_bar.png', bbox_inches='tight')


In [ ]:
# enrollment summary stats


bar_df = survival_df_clean

bar_df['enrollment_available'] = np.where(bar_df['Enrollment'].isna(), 0, 1)



bar_df = bar_df.groupby('enrollment_available').count().reset_index()



bar_df = bar_df.rename(columns={"university_state": "Number of Schools"})



bar_df['enrollment_available'] = bar_df['enrollment_available'].map({0: 'Not Available', 1: 'Data Available'})


x = bar_df['enrollment_available']
y = bar_df['Number of Schools']

fig, ax = plt.subplots()
bars = ax.bar(x, y)

ax.bar_label(bars)




plt.title('Distibution of Universities that had enrollment data')

fig.figure.savefig('enrollment_avail_bar.png', bbox_inches='tight')

In [ ]:
# enrollment summary stats


bar_df = survival_df_clean[survival_df_clean['all_year_china_donations']>0]



bar_df = bar_df.groupby(['public']).count().reset_index()



bar_df = bar_df.rename(columns={"university_state": "Number of Schools"})



bar_df['public'] = bar_df['public'].map({0: 'Private', 1: 'Public'})


x = bar_df['public']
y = bar_df['Number of Schools']

fig, ax = plt.subplots()

ax = sns.barplot(data=bar_df, x=x, y=y)

for i in ax.containers:
    ax.bar_label(i,)




plt.title('Distibution of Universities that have ever taken China money by if they are public or private')

fig.figure.savefig('china_money_pub_bar.png', bbox_inches='tight')

In [ ]:
bar_df = survival_df_clean[survival_df_clean['CI_ever']==1]

bar_df['china_ever'] = np.where(bar_df['all_year_china_donations']>0, 1, 0)



bar_df = bar_df.groupby(['china_ever']).count().reset_index()



bar_df = bar_df.rename(columns={"university_state": "Number of Schools"})



bar_df['china_ever'] = bar_df['china_ever'].map({0: 'Never taken China money', 1: 'Have taken China money'})




x = bar_df['china_ever']
y = bar_df['Number of Schools']

fig, ax = plt.subplots()

ax = sns.barplot(data=bar_df, x=x, y=y)

for i in ax.containers:
    ax.bar_label(i,)




plt.title('Distibution of Universities that have ever had a CI by if they had ever taken China money')

fig.figure.savefig('CI_ever_China_money_bar.png', bbox_inches='tight')

In [ ]:
# enrollment summary stats


bar_df = survival_df_clean[survival_df_clean['all_year_china_donations']>0]



bar_df = bar_df.groupby('CI_ever').count().reset_index()



bar_df = bar_df.rename(columns={"university_state": "Number of Schools"})



bar_df['CI_ever'] = bar_df['CI_ever'].map({0: 'Never had a CI', 1: 'Had a CI'})


x = bar_df['CI_ever']
y = bar_df['Number of Schools']

fig, ax = plt.subplots()
bars = ax.bar(x, y)

ax.bar_label(bars)




plt.title('Distibution of Universities that have ever taken China money by if they recieved a CI or not')

fig.figure.savefig('china_money_CI_bar.png', bbox_inches='tight')


In [ ]:
# check distribution of public/private schools missing enrollment data

import seaborn as sns

bar_df = survival_df_clean

bar_df['enrollment_available'] = np.where(bar_df['Enrollment'].isna(), 0, 1)



bar_df = bar_df.groupby(['enrollment_available', 'public']).count().reset_index()



bar_df = bar_df.rename(columns={"university_state": "Number of Schools"})



bar_df['enrollment_available'] = bar_df['enrollment_available'].map({0: 'Not Available', 1: 'Data Available'})


x = bar_df['enrollment_available']
y = bar_df['Number of Schools']
color = bar_df['public']

ax = sns.barplot(data=bar_df, x=x, y=y, hue=color)

for i in ax.containers:
    ax.bar_label(i,)



plt.title('Distibution of Universities that had enrollment data by Public or Private')






plt.savefig('enrollment_avail_pub_bar.png', bbox_inches='tight')


In [ ]:
bar_df

In [ ]:
# do histograms for Fed Share and Chinese money summaries
from PIL import Image



hist = survival_df_clean

hist['public/private'] = hist['public'].map({0: 'private', 1: 'public'})

priv_group =  hist[hist['public']==0]['federal_share']*100

pub_group= hist[hist['public']==1]['federal_share']*100


fig, axs = plt.subplots()

plt.hist(priv_group, label='private', alpha=.8, edgecolor='red')


plt.hist(pub_group, label='public', alpha=.8, edgecolor='green')

#fig = plt.hist(plot, normed=0)


plt.legend()


plt.xlabel("Fed Share (%)")
plt.ylabel("Count")


plt.title('Distribution of Universities by Federal Share')

# Showing the plot using plt.show()
plt.show()

fig.figure.savefig('fed_share_hist.png', bbox_inches='tight')

In [ ]:
survival_df

In [ ]:
# do histograms for Fed Share and Chinese money summaries
from PIL import Image

# tryt 

hist = survival_df_clean


priv_group =  hist[hist['public']==0]['all_year_china_donations']

pub_group= hist[hist['public']==1]['all_year_china_donations']






fig, axs = plt.subplots()

plt.hist(priv_group, label='private', alpha=.8, edgecolor='red', bins=20)


plt.hist(pub_group, label='public', alpha=.8, edgecolor='green', bins=20)

#fig = plt.hist(plot, normed=0)


plt.legend()


plt.xlabel("Total China Donations Collected ($ 100 million)")
plt.ylabel("Count")


plt.title('Distribution of Universities by Chinese Donations collected over all years')

# Showing the plot using plt.show()
plt.show()

fig.figure.savefig('china_donation_hist.png', bbox_inches='tight')


In [ ]:
# do histograms for Fed Share and Chinese money summaries
from PIL import Image

# tryt 

hist = survival_df_clean[survival_df_clean['all_year_china_donations']>0]


priv_group =  hist[hist['public']==0]['all_year_china_donations']

pub_group= hist[hist['public']==1]['all_year_china_donations']






fig, axs = plt.subplots()

plt.hist(priv_group, label='private', alpha=.8, edgecolor='red', bins=20)


plt.hist(pub_group, label='public', alpha=.8, edgecolor='green', bins=20)

#fig = plt.hist(plot, normed=0)


plt.legend()


plt.xlabel("Total China Donations Collected ($ 100 million)")
plt.ylabel("Count")


plt.title('Distribution of Universities (that have taken China Donations) by Total Chinese Donations collected over all years')

# Showing the plot using plt.show()
plt.show()

fig.figure.savefig('china_donation_taken_hist.png', bbox_inches='tight')


In [ ]:
# Number of schools by Public Private 

bar_df = survival_df_clean.groupby('public').count().reset_index()

bar_df = bar_df.rename(columns={"university_state": "Number of Schools"})



bar_df['public/private'] = bar_df['public'].map({0: 'private', 1: 'public'})


x = bar_df['public/private']
y = bar_df['Number of Schools']

fig, ax = plt.subplots()
bars = ax.bar(x, y)

ax.bar_label(bars)



plt.ylabel("Count")


plt.title('Distibution of Universities that were public or private')

fig.figure.savefig('pub_priv_bar.png', bbox_inches='tight')


In [ ]:
survival_df[survival_df['university_state']=='Alfred University_NY']


len(survival_df_clean)

In [ ]:
china_donation_log_non_norm = []


non_normalized_df = survival_df


for i in non_normalized_df['total_china_donation']:
    if i !=0:
        china_donation_log_non_norm.append(math.log(i))
    else:
        i_mod = i+1
        china_donation_log_non_norm.append(math.log(i_mod))
        
non_normalized_df['china_donation_log_non_norm'] = china_donation_log_non_norm
ols = non_normalized_df

In [ ]:
ols_CI = ols[ols['CI_ever']==1]

ols_no_CI = ols[ols['CI_ever']==0]

ols_public_CI = ols[(ols['CI_ever']==1)&(ols['public']==1) ]

ols_private_CI = ols[(ols['CI_ever']==1)&(ols['public']==0) ]



In [ ]:
y_1 =ols['china_donation_log_non_norm']

x_1= ols[['year']]

x_1 = sm.add_constant(x_1)

result_CI = sm.OLS(y_1, x_1).fit()


y_2 =ols_no_CI['china_donation_log_non_norm']
x_2 = ols_no_CI[['year']]

x_2 = sm.add_constant(x_2)

result_no_CI = sm.OLS(y_2, x_2).fit()

y_3 =ols_public_CI['china_donation_log_non_norm']

x_3 = ols_public_CI[['year']]

x_3 = sm.add_constant(x_3)

result_pub_CI = sm.OLS(y_3, x_3).fit()



y_4 =ols_private_CI['china_donation_log_non_norm']

x_4 = ols_private_CI[['year']]

x_4 = sm.add_constant(x_4)

result_priv_CI = sm.OLS(y_4, x_4).fit()



stargazer = Stargazer([result_CI, result_no_CI, result_pub_CI, result_priv_CI])




HTML(stargazer.render_latex())

In [ ]:
# OLS without interactions
from stargazer.stargazer import Stargazer
import statsmodels.api as sm
from IPython.core.display import HTML

 


stargazer = Stargazer([result_linear])

print(result_linear.summary())


HTML(stargazer.render_latex())

import seaborn as sns

ax = sns.regplot(x='year', y='china_donation_log_non_norm', order=1, label="All Universities", data=ols)

ax.legend(loc="best")


ax_no_CI = sns.regplot(x='year', y='china_donation_log_non_norm', order=1, label="No CI", data=ols_no_CI)

ax_no_CI.legend(loc="best")


ax_CI_pub = sns.regplot(x='year', y='china_donation_log_non_norm', order=1, label="Public and CI", data=ols_public_CI)

ax_CI_pub.legend(loc="best")


ax_CI_priv = sns.regplot(x='year', y='china_donation_log_non_norm', order=1, label="Private and CI", data=ols_private_CI)

ax_CI_priv.legend(loc="best")


ax_CI = sns.regplot(x='year', y='china_donation_log_non_norm', order=1, label="CI",  data=ols_CI)

ax_CI.legend(loc="best")

plt.title("Linear models of log Chinese funding over time")
plt.savefig('regressions.png')



# adding the constant term
# performing the regression
# and fitting the model
 
# printing the summary table

# make graphs from using pregrouped 

In [ ]:


# do t0 analysis with total_china_donation statistic for one university first
import itertools



uni_names_filter= survival_df[survival_df['all_year_china_donations']>0]['university_state'].unique()


normalized_df = survival_df[(survival_df['CI_ever']==1)&(survival_df['university_state'].isin(uni_names_filter))]


uni_names = normalized_df['university_state'].unique()





# make number line based on CI_opening column

school_timeline_list = []

for i in uni_names:
    school_timeline = normalized_df[normalized_df['university_state']== i]
    No_CI_years = (school_timeline['No_CI_years']-school_timeline['No_CI_years'].max()-1).drop_duplicates()
    CI_years = school_timeline['CI_survival_years'].dropna()-1
    timeline = No_CI_years.append(CI_years).to_list()
    #school_timeline['timeline'] = timeline
    school_timeline_list.append(timeline)
    




school_timeline_clean = list(itertools.chain.from_iterable(school_timeline_list))


normalized_df['timeline'] = school_timeline_clean


In [ ]:
uni_names_filter

uni_names_check= survival_df[survival_df['all_year_china_donations']>0]['university_state'].unique()


print(set(uni_names_check).difference(uni_names_filter))


survival_df[survival_df['university_state']=='Yale University_CT']

survival_df_clean[survival_df_clean['university_state']=='Arizona State University_AZ']

survival_df[survival_df['university_state']=='Arizona State University_AZ']


In [ ]:
graph_test = normalized_df[normalized_df['university_state']== 'Arizona State University_AZ']

y = graph_test['total_china_donation']
x = graph_test['timeline']


# Create a line plot


plt.plot(x, y)


plt.title("Total Chinese donations in a year for " + graph_test['schoolname'].to_list()[1] + " before and after CI opens ($)")

plt.xlabel("Number of years since opening a CI")
plt.ylabel("Total Chinese donations in a year ($ million)")

# make graphs for each university with normalized data

In [ ]:

for i in uni_names:
    graph_test = normalized_df[normalized_df['university_state']== i]
    y = graph_test['total_china_donation']
    x = graph_test['timeline']
    fig, ax = plt.subplots()
    ax.plot(x, y)
    ax.scatter(x, y)
    plt.title("Total Chinese donations in a year for " + graph_test['schoolname'].to_list()[1] + " before and after CI opens")
    plt.xlabel("Number of years since opening a CI")
    plt.ylabel("Total Chinese donations in a year ($ million)")
    fig.savefig(graph_test['schoolname'].to_list()[1] + '_donation_CI.png', bbox_inches='tight')
    plt.close(fig)

In [ ]:
china_donation_log = []


visualization_df = normalized_df
for i in normalized_df['total_china_donation']:
    if i !=0:
        china_donation_log.append(math.log(i))
    else:
        i_mod = i+1
        china_donation_log.append(math.log(i_mod))
        
visualization_df['china_donation_log'] = china_donation_log



In [ ]:
# compile graphs for 57 schools 



# try combining all these lines into one figure

y = visualization_df['china_donation_log'] 
x = visualization_df['timeline']
c =visualization_df['university_state']
fig, ax = plt.subplots()
ax.plot(x, y)
ax.scatter(x, y)
plt.title("Log Chinese donations in a year for all schools within experimental set")
plt.xlabel("timeline")
plt.ylabel("Log Chinese donations in a year")





plt.xticks(rotation=45)

break_graph = sns.lineplot(x= x, y= y, hue = c)

ax.get_legend().set_visible(False)

break_graph.figure.savefig('timeline_graph.png', bbox_inches='tight')


In [ ]:
# do similar graph for all universities except the ones with a CI AND Chinese donations
# try with only public schools
break_df = survival_df[(survival_df['public']==1)&~((survival_df['all_year_china_donations']>0) & (survival_df['CI_ever']==1))]

break_names = break_df['university_state'].unique()

# try combining all these lines into one figure

y = break_df['total_china_donation']
x = break_df['year']
c =break_df['university_state']
fig, ax = plt.subplots()
ax.plot(x, y)
ax.scatter(x, y)
plt.title("Total Chinese donations in a year for all schools outside of experimental set")
plt.xlabel("Year")
plt.ylabel("Total Chinese donations in a year ($ 10 million)")





plt.xticks(rotation=45)

break_graph = sns.lineplot(x= x, y= y, hue = c)

ax.get_legend().set_visible(False)

break_graph.figure.savefig('break_graph.png', bbox_inches='tight')



In [ ]:
survival_df



# validation
len(survival_df['university_state'].unique())-(len(break_names)+len(uni_names))

In [ ]:
ols_df_grouped = normalized_df[['university_state','schoolname','state', 'year', 'public', 'federal_share', 'total_china_donation', 'timeline']]


ols_df_grouped['timeline_grouped'] = pd.cut(ols_df_grouped['timeline'], bins=6)

ols_df_grouped['ma_china_donation'] = ols_df_grouped.groupby(['university_state','timeline_grouped'])['total_china_donation'].transform('mean')




ols_df_grouped = ols_df_grouped.rename(columns={"timeline_grouped": "T_"})
ols_df_grouped = ols_df_grouped[['university_state','schoolname','state', 'year',  'public', 'federal_share', 'ma_china_donation', 'T_']]

# fix data grouping

ols_df_grouped[ols_df_grouped['university_state']=='Alfred University_NY']

ols_df_grouped = ols_df_grouped[['university_state','schoolname','state', 'public', 'federal_share', 'ma_china_donation', 'T_']]

ols_df_grouped = ols_df_grouped.drop_duplicates()

# make graph from using this data before dummies were created

china_donation_log = []


visualization_df = ols_df_grouped
for i in visualization_df['ma_china_donation']:
    if i !=0:
        china_donation_log.append(math.log(i))
    else:
        i_mod = i+1
        china_donation_log.append(math.log(i_mod))
        
visualization_df['china_donation_log'] = china_donation_log


plt.figure(figsize=[10, 5])




ax = sns.boxplot(x="T_",
                y="china_donation_log",
                data=visualization_df).set(xlabel ="Timeline categories (year ranges)", ylabel = 'Log of moving average China donation', title = "Log average donations from China by timeline of Confucius Institute opening")


plt.savefig("ols.png") 

ols_df_grouped = pd.get_dummies(ols_df_grouped, columns=['T_'],
               drop_first=True)


In [ ]:
# summary stats tables for latex

from stargazer.stargazer import Stargazer
import statsmodels.api as sm
from IPython.core.display import HTML
summary_df = survival_df_clean[['university_state','public', 'year', 'CI_ever', 'CI_survival_years', 'CI_opening', 'federal_share', 'all_year_china_donations','Enrollment' , 'enrollment_available']]
summary_df_china_ever = survival_df_clean[['university_state','public', 'year', 'CI_ever', 'CI_survival_years', 'CI_opening', 'federal_share','Enrollment' , 'enrollment_available', 'all_year_china_donations']]


# convert these tables into latex


# summary for public
summary_stats_pub = np.round(summary_df.groupby('public').describe(percentiles = []), 2).T

# summary for enroll

summary_stats_enroll = np.round(summary_df.groupby('enrollment_available').describe(percentiles = []), 2).T

# summary for enroll and public

summary_stats_enroll_pub = np.round(summary_df.groupby(['enrollment_available', 'public']).describe(percentiles = []), 2).T


# summary for CI ever 

summary_stats_CI_ever = np.round(summary_df.groupby('CI_ever').describe(percentiles = []), 2).T

# summary for CI ever and public

summary_stats_CI_pub = np.round(summary_df.groupby(['CI_ever', 'public']).describe(percentiles = []), 2).T


summary_df_china_ever['china_ever'] = np.where((summary_df_china_ever['all_year_china_donations']>0), 1, 0)


# summary for ever taken Chinese donation

summary_stats_china_ever = np.round(summary_df_china_ever.groupby('china_ever').describe(percentiles = []), 2).T



# summary for ever taken Chinese donation and public

summary_stats_china_pub = np.round(summary_df_china_ever.groupby(['china_ever', 'public']).describe(percentiles = []), 2).T






In [ ]:



len(summary_df_china_ever[(summary_df_china_ever['china_ever']>0)&(summary_df_china_ever['CI_ever']==1)])

len(summary_df_china_ever[(summary_df_china_ever['china_ever']==1)&(summary_df_china_ever['CI_ever']==1)])


len(summary_df)


In [ ]:
# summary stats tables for latex

from stargazer.stargazer import Stargazer
import statsmodels.api as sm
from IPython.core.display import HTML
summary_df = survival_df_clean[['university_state','public', 'year', 'CI_ever', 'CI_survival_years', 'CI_opening', 'federal_share', 'all_year_china_donations','Enrollment' , 'enrollment_available']]
summary_df_china_ever = survival_df_clean[['university_state','public', 'year', 'CI_ever', 'CI_survival_years', 'CI_opening', 'federal_share','Enrollment' , 'enrollment_available', 'all_year_china_donations']]


# convert these tables into latex


# summary for public
summary_stats_pub = np.round(summary_df.groupby('public').describe(percentiles = []), 2).T

# summary for enroll

summary_stats_enroll = np.round(summary_df.groupby('enrollment_available').describe(percentiles = []), 2).T

# summary for enroll and public

summary_stats_enroll_pub = np.round(summary_df.groupby(['enrollment_available', 'public']).describe(percentiles = []), 2).T


# summary for CI ever 

summary_stats_CI_ever = np.round(summary_df.groupby('CI_ever').describe(percentiles = []), 2).T

# summary for CI ever and public

summary_stats_CI_pub = np.round(summary_df.groupby(['CI_ever', 'public']).describe(percentiles = []), 2).T


summary_df_china_ever['china_ever'] = np.where((summary_df_china_ever['all_year_china_donations']>0), 1, 0)


# summary for ever taken Chinese donation

summary_stats_china_ever = np.round(summary_df_china_ever.groupby('china_ever').describe(percentiles = []), 2).T



# summary for ever taken Chinese donation and public

summary_stats_china_pub = np.round(summary_df_china_ever.groupby(['china_ever', 'public']).describe(percentiles = []), 2).T






In [ ]:


print('\section{Summary statistics for universities by public or private}')
print(summary_stats_pub.to_latex())

print('\section{Summary statistics for universities by which ones have enrollment data in our data set}')
print(summary_stats_enroll.to_latex())
print('\section{Summary statistics for universities by which ones have enrollment data in our data set separated by public and private}')

print(summary_stats_enroll_pub.to_latex())

print('\section{Summary statistics for universities by which ones have ever had a CI}')
print(summary_stats_CI_ever.to_latex())

print('\section{Summary statistics for universities by which ones have ever had a CI separated by public and private}')

print(summary_stats_CI_pub.to_latex())


# use long table
print('\section{Summary statistics for universities by which ones have ever taken China money}')
print(summary_stats_china_ever.to_latex())


# use long table

print('\section{Summary statistics for universities by ever having taken China money separated by public or private}')

print(summary_stats_china_pub.to_latex())



In [ ]:
ma_china_donation_log = []

for i in ols_df_grouped['ma_china_donation']:
    if i !=0:
        ma_china_donation_log.append(math.log(i))
    else:
        i_mod = i+1
        ma_china_donation_log.append(math.log(i_mod))
        
ols_df_grouped['ma_china_donation_log'] = ma_china_donation_log

    

In [ ]:
# OLS without interactions
from stargazer.stargazer import Stargazer
import statsmodels.api as sm
from IPython.core.display import HTML

 
y =ols_df_grouped['ma_china_donation_log']

x = ols_df_grouped[['public','T__(-16.0, -10.0]','T__(-10.0, -4.0]','T__(-4.0, 2.0]','T__(2.0, 8.0]','T__(8.0, 14.0]']]

x = sm.add_constant(x)



# adding the constant term
 
# performing the regression
# and fitting the model
result_1 = sm.OLS(y, x).fit()
 
# printing the summary table

# make graphs from using pregrouped 

In [ ]:

# OLS with interactions

import statsmodels.api as sm

 
y =ols_df_grouped['ma_china_donation_log']

x = ols_df_grouped[['public','T__(-16.0, -10.0]','T__(-10.0, -4.0]','T__(-4.0, 2.0]','T__(2.0, 8.0]','T__(8.0, 14.0]']]

x = sm.add_constant(x)

x['int_1'] = x['public']*x['T__(2.0, 8.0]']
x['int_2'] = x['public']*x['T__(8.0, 14.0]']

# adding the constant term
 
# performing the regression
# and fitting the model
result_3 = sm.OLS(y, x).fit()
 


In [ ]:

# Public OLS regression

import statsmodels.api as sm

data_pub = ols_df_grouped[ols_df_grouped['public']==1]
y =data_pub['ma_china_donation_log']
x = data_pub[['T__(-16.0, -10.0]','T__(-10.0, -4.0]','T__(-4.0, 2.0]','T__(2.0, 8.0]','T__(8.0, 14.0]']]

x = sm.add_constant(x)


# adding the constant term
 
# performing the regression
# and fitting the model
result_4 = sm.OLS(y, x).fit()
 


In [ ]:

# Private OLS regression

import statsmodels.api as sm

data_priv = ols_df_grouped[ols_df_grouped['public']==0]
y =data_priv['ma_china_donation_log']
x = data_priv[['T__(-16.0, -10.0]','T__(-10.0, -4.0]','T__(-4.0, 2.0]','T__(2.0, 8.0]','T__(8.0, 14.0]']]

x = sm.add_constant(x)


# adding the constant term
 
# performing the regression
# and fitting the model
result_5 = sm.OLS(y, x).fit()
 





In [ ]:
# remove duplicate rows form cross tab

import seaborn as sns


ols_df_binary = normalized_df[['university_state','schoolname','state', 'year', 'public', 'federal_share', 'total_china_donation', 'timeline']]

def timeline_sep(x):
    if 0 < x:
        return 't0_after'
    elif x==0:
        return 't0'
    else:
        return 't0_before'

    
ols_df_binary['timeline_sep'] = ols_df_binary['timeline'].apply(timeline_sep)




ols_df_binary['ma_china_donation'] = ols_df_binary.groupby(['university_state','timeline_sep'])['total_china_donation'].transform('mean')



ma_china_donation_log_bin = []

for i in ols_df_binary['ma_china_donation']:
    if i !=0:
        ma_china_donation_log_bin.append(math.log(i))
    else:
        i_mod = i+1
        ma_china_donation_log_bin.append(math.log(i_mod))
        
ols_df_binary['ma_china_donation_log'] = ma_china_donation_log_bin



ols_df_binary = ols_df_binary[['university_state','schoolname','state', 'year',  'public', 'federal_share', 'ma_china_donation_log', 'timeline_sep']]

# fix data grouping


ols_df_binary = ols_df_binary[['university_state','schoolname','state', 'public', 'federal_share', 'ma_china_donation_log', 'timeline_sep']]

plt.figure(figsize=[10, 5])


ax = sns.boxplot(x="timeline_sep",
                y="ma_china_donation_log",
                data=ols_df_binary).set(xlabel ="Timeline", ylabel = 'Log of moving average China donation', title = "Log average donations from China by before or after opening a Confucius Institute")


plt.savefig("ols_binary.png") 

ols_df_binary = pd.get_dummies(ols_df_binary, columns=['timeline_sep'],
               drop_first=True)




In [ ]:
# Binary OLS regression
import statsmodels.api as sm

 
y = ols_df_binary[['ma_china_donation_log']]

x = ols_df_binary[['timeline_sep_t0_after']]


x['t0_after'] = x[['timeline_sep_t0_after']]

x = x['t0_after']
 
# adding the constant term
x = sm.add_constant(x)
 
# performing the regression
# and fitting the model
result_2 = sm.OLS(y, x).fit()
 
# printing the summary table

stargazer = Stargazer([result_1, result_2, result_3, result_4, result_5])




HTML(stargazer.render_latex())


In [ ]:
# make dummy variable with timeline
# pivot data wide with timeline variable

ols_df = normalized_df[['university_state','schoolname','state', 'year', 'public', 'federal_share', 'total_china_donation', 'timeline']]

ols_df = ols_df.rename(columns={"timeline": "index"})

ols_df = (
    pd.crosstab([ols_df["university_state"], ols_df["schoolname"], ols_df["state"], ols_df["year"], ols_df["public"],  ols_df["total_china_donation"]], ols_df["index"], )
    .gt(0)
    .astype(int)
    .reset_index()
)





ols_df.columns = [f'T_{i}' if i not in ['university_state','schoolname','state', 'year', 'public', 'federal_share', 'total_china_donation'] else f'{i}' for i in ols_df.columns]


# try grouping by column values to make groups
# calcualte moving average using grouped data
# try making dummy variable by if a variable is in a certain range of years ie., data in T-22, T_-21, T-19 or etc

In [ ]:


#ols_df.columns[6:]


In [ ]:
# modify this OLS code

import statsmodels.api as sm

 
y = ols_df[['total_china_donation']]/1000000

x = ols_df[['public', 'T_-22.0', 'T_-21.0', 'T_-20.0', 'T_-19.0', 'T_-18.0', 'T_-17.0',
       'T_-16.0', 'T_-15.0', 'T_-14.0', 'T_-13.0', 'T_-12.0', 'T_-11.0',
       'T_-10.0', 'T_-9.0', 'T_-8.0', 'T_-7.0', 'T_-6.0', 'T_-5.0', 'T_-4.0',
       'T_-3.0', 'T_-2.0', 'T_-1.0', 'T_0.0', 'T_1.0', 'T_2.0', 'T_3.0',
       'T_4.0', 'T_5.0', 'T_6.0', 'T_7.0', 'T_8.0', 'T_9.0', 'T_10.0',
       'T_11.0', 'T_12.0', 'T_13.0', 'T_14.0']]


 
# adding the constant term
x = sm.add_constant(x)
 
# performing the regression
# and fitting the model
result = sm.OLS(y, x).fit()
 
# printing the summary table
print(result.summary())

In [ ]:
survival_df_clean = survival_df_clean.rename({'No_CI_years': 'survival_years'}, axis=1)



survival_df_clean['status'] = survival_df_clean['status'].astype('bool')




survival_df_clean

%matplotlib inline
import matplotlib.pyplot as plt

import sksurv

from sksurv.nonparametric import kaplan_meier_estimator

time, survival_prob, conf_int = kaplan_meier_estimator(
    survival_df_clean["status"], survival_df_clean["survival_years"], conf_type="log-log"
)
plt.step(time, survival_prob, where="post")
plt.fill_between(time, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.ylim(0, 1)
plt.ylabel("est. probability of not having a CI $\hat{S}(t)$")
plt.xlabel("time in years $t$")
plt.title('Probability of not having a Confucius Institute over time')


plt.savefig('CI_survival.png', bbox_inches='tight')



In [ ]:
#survival_df_clean['survival_odds'] = survival_df_clean['survival_years']/max(survival_df_clean['survival_years'])
survival_df_clean['CI_ever'] = survival_df_clean['CI_ever'].astype('bool')


china_donations = survival_df_clean[survival_df_clean['cumulative_china_donations'].notna()]



# try this https://stats.stackexchange.com/questions/496329/converting-survival-analysis-by-a-continuous-variable-to-categorical-so-as-to-fi
CI_ever, survival_prob, conf_int = kaplan_meier_estimator(
    china_donations["status"], china_donations["cumulative_china_donations"], conf_type="log-log"
)
plt.step(CI_ever, survival_prob, where="post")
plt.fill_between(CI_ever, conf_int[0], conf_int[1], alpha=0.25, step="post")
plt.ylim(0, 1)
plt.ylabel("est. probability of not having a CI $\hat{S}(t)$")
plt.xlabel("Cumulative Chinese dollar donations recieved in 100,000,000's")
plt.title('Probability of not having a Confucius Institute compared to Chinese donations')

plt.savefig('chinese_donation_survival.png', bbox_inches='tight')



In [ ]:
# do this graph with Chinese donations on the X axis
# See what happens with Chinese donations once Confucius Institutes close
# Try clumping donation years together to deal with NaN values
# Has CI funding dropped a lot after CI closed?
# Has school relaunched Confucius Institutes with rebrand
# Check Katherine data to see which CI's survived but in different form

for public in (0, 1):
    mask_treat = survival_df_clean["public"] == public
    time_treatment, survival_prob_treatment = kaplan_meier_estimator(
        survival_df_clean["CI_ever"][mask_treat],
        survival_df_clean["survival_years"][mask_treat])
    
    plt.step(time_treatment, survival_prob_treatment, where="post",
             label="Public = %s" % public)

plt.ylabel("est. probability of not having a CI $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.title('Survival rate of not having a Confucius Institute for public and private schools')
plt.legend(loc="best")

plt.savefig('pub_vs_priv_survival.png', bbox_inches='tight')


In [ ]:
# make new variable for cumulative China donations that stops once the University opens a Confucisus Institute
# use this new variable as a time scale


for public in (0, 1):
    mask_treat = survival_df_clean["public"] == public
    time_treatment, survival_prob_treatment = kaplan_meier_estimator(
        survival_df_clean["status"][mask_treat],
        survival_df_clean["cumulative_china_donations"][mask_treat])
    
    plt.step(time_treatment, survival_prob_treatment, where="post",
             label="Public = %s" % public)

plt.ylabel("est. probability of survival $\hat{S}(t)$")
plt.xlabel("Cumulative Chinese dollar donations in 100,000,000's")
plt.title('Survival rate of not having a Confucius Institute for public and private schools')
plt.legend(loc="best")

plt.savefig('pub_vs_priv_money_survival.png', bbox_inches='tight')


In [ ]:
survival_df_clean['avg_china_donations']= survival_df_clean['all_year_china_donations']/(2020-1995)
survival_df_clean['avg_china_donations'].hist()


plt.title('Histogram of yearly donations')

plt.savefig('donation_histogram.png', bbox_inches='tight')


In [ ]:
# make survival grah with 0 being cutoff for Chinese donations
# make binary variable for china donation

survival_df_clean['china_ever'] = np.where(survival_df_clean['avg_china_donations'] > 0, 1, 0)


for china_ever in (0, 1):
    mask_treat = survival_df_clean["china_ever"] == china_ever
    time_treatment, survival_prob_treatment = kaplan_meier_estimator(
        survival_df_clean["status"][mask_treat],
        survival_df_clean["survival_years"][mask_treat])
    
    plt.step(time_treatment, survival_prob_treatment, where="post",
             label="china_ever = %s" % china_ever)

plt.ylabel("est. probability of not having a CI $\hat{S}(t)$")
plt.xlabel("time $t$")
plt.title('Probability of not having a Confucius Institute by having ever taken a donation from China')
plt.legend(loc="best")

plt.savefig('china_donation_survival.png', bbox_inches='tight')



In [ ]:
# make graph where you groupby CI ever and not CI

# find mean closing year
# see how much donations change past mean closing year




master_df_enroll['CI_survival'] = master_df_enroll['CI_open'].diff()

master_df_enroll[master_df_enroll['schoolname']=='Arizona State University']



CI_close_year = master_df_clean[master_df_clean['CI_survival']==-1]

CI_close_year['CI_close_year']= 1

master_df_enroll= master_df_enroll.merge(CI_close_year, how = 'left')

master_df_enroll['CI_close_year']=  master_df_enroll['CI_close_year'].fillna(0)

# group by close year and find mean close year
# check how average donations change before and after mean close year


In [ ]:
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.preprocessing import OneHotEncoder
from lifelines import CoxPHFitter


#split data into data_x and data_y 

survival_model = survival_df_clean.dropna()



# try binary version of donation vs non China donation for CI survival
# Look at overall distribution of funding and draw line for significant vs non-sihgnficant funding
# for every metric (average annual Chinese donation), check the distirbtuion with a histogram for Chinese donations
# use bottom point of histogram to see the cutoff to make two distinctive
# also figure out how to interpret Cox PH regression
# check distribution of public vs private universities in data set
# look up some literature on confucius institues in public vs private
# look at literature on Chinese influence in public vs private universties
# look at NAS studies on Confucius Institue work




In [ ]:
survival_model_public = survival_df_clean[survival_df_clean['public'].notna()]

cph = CoxPHFitter()
survival_years = cph.fit(survival_model_public, duration_col = 'survival_years', event_col = 'status', formula= "public + all_year_china_donations + all_year_foreign_donations")
cph.print_summary()


print('Being a public university increases the hazard for getting a CI by a factor of 4.42, or ',(4.42-1)*100,'%' )



In [ ]:
# install stargazer
from stargazer.stargazer import Stargazer

cph = CoxPHFitter()
survival_years = cph.fit(survival_model, duration_col = 'survival_years', event_col = 'status', formula= "public + all_year_federal10 + all_year_tot_rev_wo_auxother_sum+all_year_china_donations+all_year_foreign_donations")
cph.print_summary()


print('Being a public university increases the hazard for getting a CI by a factor of 4.19, or ',(4.19-1)*100,'%' )





In [ ]:

cph = CoxPHFitter()
donation_cph = cph.fit(survival_model, duration_col = 'cumulative_china_donations', event_col = 'status', formula= "public + all_year_federal10 + all_year_tot_rev_wo_auxother_sum+all_year_foreign_donations")
donation_cph.print_summary()


print('Being a public university increases the hazard for getting a CI as Chinese donation money increases by a factor of 3.68, or ',(3.68-1)*100,'%' )

In [ ]:
pie_df = survival_df_clean.groupby('public').count().reset_index()

pie_df = pie_df[['public','university_state']]


pie_df = pie_df.rename(columns={"university_state": "Number of Schools"})



pie_df['school_type'] = pie_df['public'].map({0: 'private', 1: 'public'})


labels = pie_df['school_type']
sizes = pie_df['Number of Schools']




fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', pctdistance=1.25, labeldistance=.6)

plt.title('Share of Public and Private Universities')

fig.figure.savefig('public_private_pie.png', bbox_inches='tight')

pie_df


In [ ]:
CI_universities = survival_df_clean[survival_df_clean['CI_ever']==1]


pie_df = CI_universities.groupby('public').count().reset_index()

pie_df = pie_df[['public','university_state']]


pie_df = pie_df.rename(columns={"university_state": "Number of Schools"})



pie_df['school_type'] = pie_df['public'].map({0: 'private', 1: 'public'})


labels = pie_df['school_type']
sizes = pie_df['Number of Schools']




fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', pctdistance=1.25, labeldistance=.6)

plt.title('Share of Public and Private Universities that ever had a CI')

fig.figure.savefig('public_private_CI_pie.png', bbox_inches='tight')

pie_df

In [ ]:
No_CI_China = survival_df_clean.loc[(survival_df_clean['CI_ever'] == False) & (survival_df_clean['cumulative_china_donations'] >0)]




pie_df = No_CI_China.groupby('public').count().reset_index()

pie_df = pie_df[['public','university_state']]


pie_df = pie_df.rename(columns={"university_state": "Number of Schools"})



pie_df['school_type'] = pie_df['public'].map({0: 'private', 1: 'public'})


labels = pie_df['school_type']
sizes = pie_df['Number of Schools']




fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, autopct='%1.1f%%', pctdistance=1.25, labeldistance=.6)

plt.title('Share of Public and Private Universities that never had a CI but took China money')

fig.figure.savefig('public_private_No_CI_China_pie.png', bbox_inches='tight')

pie_df



In [ ]:
No_CI_China['university_state'].unique()

In [ ]:
pie_df

In [ ]:
survival_df_clean

In [ ]:

summary_stats = np.round(master_df_clean.describe(), 2).T[['count','mean', 'std', 'min', 'max']]

summary_stats_CI = np.round(master_df_clean[master_df_clean['CI_ever'] == 1].describe(), 2).T[['count','mean', 'std', 'min', 'max']]


summary_stats_no_CI = np.round(master_df_clean[master_df_clean['CI_ever'] == 0].describe(), 2).T[['count','mean', 'std', 'min', 'max']]


CI_survival = survival_df_clean.loc[(survival_df_clean['CI_ever'] == True)]
    
    

summary_stats_survival = np.round(survival_df_clean.describe(), 2).T[['count','mean', 'std', 'min', 'max']]

CI_survival_summary = np.round(CI_survival.describe(), 2).T[['count','mean', 'std', 'min', 'max']]


No_CI_China_summary = np.round(No_CI_China.describe(), 2).T[['count','mean', 'std', 'min', 'max']]



descriptive_stats_list= [summary_stats, summary_stats_CI, summary_stats_no_CI]


descriptive_stats_names=  ['summary_stats', 'summary_stats_CI', 'summary_stats_no_CI']

import matplotlib.pyplot as plt
from pandas.plotting import table



for i, j in zip(descriptive_stats_list, descriptive_stats_names):
    plot = plt.subplot(111, frame_on=False)


    #remove axis
    plot.xaxis.set_visible(False) 
    plot.yaxis.set_visible(False) 

#create the table plot and position it in the upper left corner
    table(plot, i, loc='upper right')
    plt.savefig(j +'.png', bbox_inches='tight')


In [ ]:
No_CI_survival = survival_df_clean.loc[(survival_df_clean['CI_ever'] == False)]
No_CI_survival_summary = np.round(No_CI_survival.describe(), 2).T[['count','mean', 'std', 'min', 'max']]


In [ ]:


descriptive_survival_list= [No_CI_China_summary, summary_stats_survival, CI_survival_summary, No_CI_survival_summary]


descriptive_survival_names=  ['No_CI_China_summary', 'summary_stats_survival', 'CI_survival_summary', 'No_CI_survival_summary']

import matplotlib.pyplot as plt
from pandas.plotting import table



for i, j in zip(descriptive_survival_list, descriptive_survival_names):
    plot = plt.subplot(111, frame_on=False)


    #remove axis
    plot.xaxis.set_visible(False) 
    plot.yaxis.set_visible(False) 

#create the table plot and position it in the upper left corner
    table(plot, i, loc='upper right')
    plt.savefig(j +'.png', bbox_inches='tight')

In [ ]:
No_CI_survival_summary['CI_ever'].unique()

In [ ]:
# Graph output settings
import matplotlib.pyplot as plt




# put school name here
schoolname ="Arizona State University"

# put state code here

st = "AZ"

school_for_analysis = schoolname+'_'+st




year = 2006


master_df_clean[master_df_clean['schoolname']=='Arizona State University']


In [ ]:
example_graph = master_df_clean.groupby(['state'])['china_donation_over_federal'].mean().reset_index()

example_graph = example_graph.sort_values('china_donation_over_federal',ascending = False).head(10)

# filter this graph by top 5 states
x=example_graph['state']

y = example_graph['china_donation_over_federal']


plt.title('Top ten states that have the highest mean proportion of Chinese donations compared to federal funds')


plt.xticks(rotation=0, )

plt.tick_params(axis='x', which='major', labelsize=7)


example_donation_plot = sns.barplot(x= x, y= y)


example_donation_plot.figure.savefig('Chinese_donation_reliance_by_state.png', bbox_inches='tight')


# covert this data into a bar graph

master_df_clean[master_df_clean['total_foreign_donation']==0]

In [ ]:
example_graph = master_df_clean.groupby(['public', 'year'])['china_donation_over_federal'].mean().reset_index()

x=example_graph['year']

y = example_graph['china_donation_over_federal']
c =example_graph['public']

plt.title('Time series for public vs private proportion of Chinese donations to federal funds')


plt.xticks(rotation=45)


example_donation_plot = sns.barplot(x= x, y= y, hue= c)


example_donation_plot.figure.savefig('Chinese_donation_reliance.png', bbox_inches='tight')


# covert this data into a bar graph

In [ ]:
china_donation_2016 = master_df_clean[(master_df_clean['year']==2016)&(master_df_clean['public']==0)&(master_df_clean['china_donation'].notna())]

china_donation_2016.sort_values(by=['china_donation_over_federal'], ascending=False).head(10)

In [ ]:
# fix this pie chart!!
example_graph = master_df_clean.groupby(['public', 'CI_ever']).size().reset_index()


example_graph = example_graph.rename(columns = {0: 'count'})

x= example_graph['public']

y = example_graph['count']
c =example_graph['CI_ever']

CI_pub_plot = sns.barplot(x= x, y= y, hue= c)

plt.title('Number of public and private Universities that have ever had a CI')


CI_pub_plot.figure.savefig('CI_pub_plot.png', bbox_inches='tight')



In [ ]:


example_graph = master_df_clean[master_df_clean['university_state'].isin([school_for_analysis])]


x=example_graph['year']

y = example_graph['total_china_donation']
c =example_graph['CI_open']

plt.title('Time series for '+schoolname+' CI open years and Chinese donations')


plt.xticks(rotation=45)


example_donation_plot = sns.barplot(x= x, y= y, hue= c)


example_donation_plot.figure.savefig('example_donation_plot.png', bbox_inches='tight')



In [ ]:
import matplotlib.pyplot as plt

example_graph = master_df_clean[master_df_clean['university_state'].isin([school_for_analysis])]


x=example_graph['year']

y = example_graph['china_donation_over_federal']
c =example_graph['CI_open']

plt.title('Time series for '+schoolname+ ' CI open years and ratio of Chinese donations to Federal Funds')


plt.xticks(rotation=45)


example_donation_plot = sns.barplot(x= x, y= y, hue= c)

example_donation_plot.figure.savefig('school_ratio_federal_funds.png', bbox_inches='tight')



In [ ]:
len(master_df_clean[master_df_clean['r']=='R']['university_state'].unique())

In [ ]:
example_replacement =  master_df_clean[master_df_clean['r']=='R']['university_state'].unique()[21]




example_graph = master_df_clean[master_df_clean['university_state'].isin([example_replacement])]

replacement_name  = example_graph['schoolname'].unique()[0]


x=example_graph['year']

y = example_graph['total_china_donation']
c =example_graph['CI_close_year']
schools = example_graph['schoolname']

plt.title('Time series for '+replacement_name+ ' total Chinese donations with a mark when the CI closed')


plt.xticks(rotation=45)

example_donation_plot = sns.lineplot(x= x, y= y)


example_donation_plot = sns.scatterplot(x= x, y= y, hue= c, s = 100)


example_donation_plot.figure.savefig('CI_replaced_graph.png', bbox_inches='tight')




In [ ]:


example_graph = master_df_clean[master_df_clean['university_state'].isin([school_for_analysis])]


x=example_graph['year']

y = example_graph['total_china_donation']
c =example_graph['CI_close_year']
schools = example_graph['schoolname']

plt.title('Time series for '+schoolname+ ' total Chinese donations with a mark when the CI closed')


plt.xticks(rotation=45)

example_donation_plot = sns.lineplot(x= x, y= y, hue = schools) 


example_donation_plot = sns.scatterplot(x= x, y= y, hue= c, s = 100)


example_donation_plot.figure.savefig('CI_closed_graph.png', bbox_inches='tight') 




In [ ]:
example_graph = master_df_clean[master_df_clean['university_state'].isin([school_for_analysis])]

 
x=example_graph['year']

y = example_graph['CI_open']
c =example_graph['china_donation']

plt.title('Time series for '+schoolname+ ' CI open years and if they recieved Chinese donations that year')



plt.xticks(rotation=45)




example_CI_binary_plot = sns.scatterplot(x= x, y= y, hue= c)


example_CI_binary_plot.figure.savefig('example_CI_binary_plot.png', bbox_inches='tight')


In [ ]:
master_df_clean[master_df_clean['schoolname']=="Alabama A&M University"]

df[df['schoolname']=="Harvard University"]




df[df['schoolname']=="Harvard University"]

master_df_clean[master_df_clean['schoolname']=="Harvard University"]


df[(df['schoolname']=="Arizona State University") & (df['receipt_year']==1996)]

 

df[(df['schoolname']=="Arizona State University")]

master_df_clean[master_df_clean['schoolname']=="Stanford University"]


master_df_clean[master_df_clean['schoolname']== "Harvard University"]

In [ ]:
survival_df_clean.to_stata('CI_survival_data_cleaned.dta')  


master_df_clean.to_stata('school_year_data_cleaned.dta')  


In [ ]:
master_df_clean.dtypes

